# 2019-nCov Analysis, $R_0$ estimation
### Affan Shoukat

Analysis of the results from the `EpiEstim` package. After the `epi-estim-analysis.R` file is run, this script can be used to read the saved files and summarize the data. This can be done in any programming language, since it's just reading data.

In [2]:
# load packages, read the data, and melt it into long format
using Distributions, DataFrames, CSV, Query, VegaLite, LsqFit, Plots, PrettyTables, StatsPlots, Interpolations
using Base.Iterators, Random
pyplot()
#PyPlot.svg(true)

Plots.PyPlotBackend()

In [25]:
rosumm = CSV.read("../data/results_feb6_paperv2/baseline_rvalues_summary.csv", datarow=2, header=["time", "Rmean", "Rlow", "Rhi"])
aa = rosumm |> @filter(_.time in [29, 39, 46, 53, 60, 64]) |> @map({_.time, rmean = round(_.Rmean, digits=2), rlow = round(_.Rlow, digits=2), Rhi = round(_.Rhi, digits=2)}) |> DataFrame
pretty_table(aa, crop=:none)

rosumm = CSV.read("../data/results_feb6_paperv2/ur20_rvalues_summary.csv", datarow=2, header=["time", "Rmean", "Rlow", "Rhi"])
aa = rosumm |> @filter(_.time in [29, 39, 46, 53, 60, 64]) |> @map({_.time, rmean = round(_.Rmean, digits=2), rlow = round(_.Rlow, digits=2), Rhi = round(_.Rhi, digits=2)}) |> DataFrame
pretty_table(aa, crop=:none)

rosumm = CSV.read("../data/results_feb6_paperv2/ur50_rvalues_summary.csv", datarow=2, header=["time", "Rmean", "Rlow", "Rhi"])
aa = rosumm |> @filter(_.time in [29, 39, 46, 53, 60, 64]) |> @map({_.time, rmean = round(_.Rmean, digits=2), rlow = round(_.Rlow, digits=2), Rhi = round(_.Rhi, digits=2)}) |> DataFrame
pretty_table(aa, crop=:none)

rosumm = CSV.read("../data/results_feb6_paperv2/ur90_rvalues_summary.csv", datarow=2, header=["time", "Rmean", "Rlow", "Rhi"])
aa = rosumm |> @filter(_.time in [29, 39, 46, 53, 60, 64]) |> @map({_.time, rmean = round(_.Rmean, digits=2), rlow = round(_.Rlow, digits=2), Rhi = round(_.Rhi, digits=2)}) |> DataFrame
pretty_table(aa, crop=:none)


#december 29, january 8, january 22 feb 8
    

┌───────┬─────────┬─────────┬─────────┐
│  time │   rmean │    rlow │     Rhi │
│ Int64 │ Float64 │ Float64 │ Float64 │
├───────┼─────────┼─────────┼─────────┤
│  29.0 │    2.66 │    1.78 │    3.82 │
│  39.0 │    2.53 │     1.9 │    3.35 │
│  46.0 │    2.51 │    1.97 │    3.23 │
│  53.0 │     2.5 │    1.99 │    3.18 │
│  60.0 │    2.48 │    1.99 │    3.12 │
│  64.0 │     2.5 │    2.01 │    3.15 │
└───────┴─────────┴─────────┴─────────┘
┌───────┬─────────┬─────────┬─────────┐
│  time │   rmean │    rlow │     Rhi │
│ Int64 │ Float64 │ Float64 │ Float64 │
├───────┼─────────┼─────────┼─────────┤
│  29.0 │    2.57 │    1.76 │    3.64 │
│  39.0 │    2.53 │    1.93 │    3.33 │
│  46.0 │    2.53 │    1.98 │    3.25 │
│  53.0 │     2.5 │    1.99 │    3.19 │
│  60.0 │    2.49 │     2.0 │    3.16 │
│  64.0 │    2.52 │    2.01 │    3.19 │
└───────┴─────────┴─────────┴─────────┘
┌───────┬─────────┬─────────┬─────────┐
│  time │   rmean │    rlow │     Rhi │
│ Int64 │ Float64 │ Float64 │ Float64 │


In [7]:
inc = CSV.read("../data/results_feb6_paperv2/baseline_incidence.csv", datarow=2)
pretty_table(inc, crop=:none)

┌───────┬─────────┬─────────┬─────────┬──────────────────┐
│  time │       m │      md │     qlo │              qhi │
│ Int64 │ Float64 │ Float64 │ Float64 │          Float64 │
├───────┼─────────┼─────────┼─────────┼──────────────────┤
│  12.0 │     0.0 │     0.0 │     0.0 │              0.0 │
│  13.0 │     0.0 │     0.0 │     0.0 │              0.0 │
│  14.0 │     0.0 │     0.0 │     0.0 │              0.0 │
│  15.0 │     0.0 │     0.0 │     0.0 │              0.0 │
│  16.0 │   0.992 │     1.0 │     0.0 │              3.0 │
│  17.0 │   1.024 │     1.0 │     0.0 │              4.0 │
│  18.0 │   0.994 │     1.0 │     0.0 │              3.0 │
│  19.0 │   0.944 │     1.0 │     0.0 │              3.0 │
│  20.0 │   0.966 │     1.0 │     0.0 │              3.0 │
│  21.0 │   1.052 │     1.0 │     0.0 │ 3.52499999999998 │
│  22.0 │   0.994 │     1.0 │     0.0 │              3.0 │
│  23.0 │   0.988 │     1.0 │     0.0 │              3.0 │
│  24.0 │   0.976 │     1.0 │     0.0 │              3.0

In [21]:

for x in ("baseline", "ur20", "ur50", "ur90")
    finalsize = CSV.read("../data/results_feb6_paperv2/$(x)_finalsize.csv", datarow=2, header=["seed", "jan29", "feb10"])
    println(mean(finalsize.feb10))
    println(quantile(finalsize.feb10, [0.025, 0.975]))
    #println(quantile(finalsize[:, Symbol("final.size")], 0.975))    
end


5144.964
[4299.65, 6071.15]
6456.666
[5423.225, 7587.999999999999]
25876.674
[21599.6, 29988.25]
51868.838
[42735.55, 60717.25]


4939.0
7100.674999999999
